In [2]:
import threading
import time
import queue
import numpy as np
from datetime import datetime


In [3]:
# Buffers for each sensor stream (holds tuples of (timestamp, value))
camera_queue = queue.Queue(maxsize=10)  # Stores last 10 predictions
gyro_queue = queue.Queue(maxsize=20)  # Faster sensor, larger buffer
spo2_queue = queue.Queue(maxsize=5)  # Slowest sensor

# Lock for thread safety
lock = threading.Lock() 


In [4]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import pickle
import joblib
import time
import csv
import os
from datetime import datetime
from queue import Queue
import threading

# Load the scaler and KNN model
scaler_path = "C:\\Users\\HP\\Documents\\DSGP\\CM2603-Data_Science_Project-G20\\models\\camera\\Fall_Detection\\KNN_model\\scaler.pkl"
model_path = "C:\\Users\\HP\\Documents\\DSGP\\CM2603-Data_Science_Project-G20\\models\\camera\\Fall_Detection\\KNN_model\\knn_model.pkl"

scaler = joblib.load(scaler_path)
with open(model_path, "rb") as model_file:
    knn_model = pickle.load(model_file)

# MediaPipe Pose Setup
mp_pose = mp.solutions.pose
non_face_indices = list(range(11, 33))

# Shared Queue & Lock for Multi-threading
camera_queue = Queue(maxsize=100)
lock = threading.Lock()

# Define log file path
log_file = "camera_log.csv"

# Ensure the log file exists and has headers
if not os.path.exists(log_file):
    with open(log_file, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Prediction"])  # Column headers

def process_camera():
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Camera not accessible.")
        return

    with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame.")
                break

            image = cv.resize(frame, (640, 480))
            image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            prediction = 0  # Default: No fall detected
            if results.pose_landmarks:
                height, width, _ = image.shape
                landmarks = results.pose_landmarks.landmark
                filtered_landmarks = [landmarks[i] for i in non_face_indices]

                x_coords = [lm.x * width for lm in filtered_landmarks]
                y_coords = [lm.y * height for lm in filtered_landmarks]
                visibilities = [lm.visibility for lm in filtered_landmarks]
                
                features_input = np.array([
                    np.mean(x_coords), np.mean(y_coords), np.mean(visibilities),
                    max(x_coords) - min(x_coords), max(y_coords) - min(y_coords)
                ])
                features_input_scaled = scaler.transform([features_input])

                prediction = knn_model.predict(features_input_scaled)[0]  # 1 = Fall, 0 = Non-Fall
            
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
            with lock:
                if camera_queue.full():
                    camera_queue.get()  # Remove the oldest entry
                camera_queue.put((timestamp, prediction))
                
                # Append to log file
                with open(log_file, "a", newline="") as file:
                    writer = csv.writer(file)
                    writer.writerow([timestamp, prediction])
            
            time.sleep(1 / 30)  # Maintain ~30 FPS

    cap.release()
    cv.destroyAllWindows()


In [5]:
def process_gyro():
    while True:
        # Simulating an accelerometer & gyroscope prediction
        prediction = np.random.choice([0, 1], p=[0.9, 0.1])
        timestamp = datetime.now()

        with lock:
            if gyro_queue.full():
                gyro_queue.get()
            gyro_queue.put((timestamp, prediction))

        time.sleep(1 / 50)  # Simulating 50 Hz (~20ms per reading)


In [6]:
def process_spo2():
    while True:
        # Simulating a pulse oximeter prediction
        prediction = np.random.choice([0, 1], p=[0.95, 0.05])  # 5% chance of abnormal reading
        timestamp = datetime.now()

        with lock:
            if spo2_queue.full():
                spo2_queue.get()
            spo2_queue.put((timestamp, prediction))

        time.sleep(1)  # Runs at 1 Hz (every second)


In [7]:
def fusion_algorithm():
    while True:
        with lock:
            # Get the most recent values
            cam_pred = camera_queue.queue[-1] if not camera_queue.empty() else (datetime.now(), 0)
            gyro_pred = gyro_queue.queue[-1] if not gyro_queue.empty() else (datetime.now(), 0)
            spo2_pred = spo2_queue.queue[-1] if not spo2_queue.empty() else (datetime.now(), 0)

        # Extract values
        cam_time, cam_value = cam_pred
        gyro_time, gyro_value = gyro_pred
        spo2_time, spo2_value = spo2_pred

        # Weighted Sum Calculation
        weighted_sum = (4 * cam_value) + (2 * gyro_value) + (1 * spo2_value)

        # Decision thresholds 
        if weighted_sum >= 4:
            final_prediction = "🚨 Fall Detected"
        elif weighted_sum == 3 and cam_value == 0:
            final_prediction = "⚠️ Risk of Falling"
        else:
            final_prediction = "✅ Non-Fall"

        print(f"[{datetime.now().strftime('%H:%M:%S')}] Camera({cam_value}) Gyro({gyro_value}) SpO2({spo2_value}) → Prediction: {final_prediction}")

        time.sleep(0.1)  # Fusion runs every 100ms


In [ ]:
if __name__ == "__main__":
    try:
        # Create threads for each sensor
        camera_thread = threading.Thread(target=process_camera)
        gyro_thread = threading.Thread(target=process_gyro)
        spo2_thread = threading.Thread(target=process_spo2)
        fusion_thread = threading.Thread(target=fusion_algorithm)

        # Start all threads
        camera_thread.start()
        gyro_thread.start()
        spo2_thread.start()
        fusion_thread.start()

        # Wait for user to exit
        while True:
            time.sleep(1)

    except KeyboardInterrupt:
        print("\n[INFO] Stopping all threads...")

        # Set stop event to signal all threads to stop
        stop_event.set()

        # Wait for threads to finish
        camera_thread.join()
        gyro_thread.join()
        spo2_thread.join()
        fusion_thread.join()

        print("[INFO] All threads stopped. Exiting.")

        # Exit cleanly
        sys.exit(0)

[21:10:50] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:50] Camera(0) Gyro(1) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:50] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:50] Camera(0) Gyro(1) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:50] Camera(0) Gyro(1) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(1) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(1) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:51] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:52] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:5

c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[21:10:52] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:52] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall


c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[21:10:52] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:52] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:10:52] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall


...

c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Proje

[21:11:35] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:11:35] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:11:35] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall


c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[21:11:35] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:11:35] Camera(0) Gyro(1) SpO2(0) → Prediction: ✅ Non-Fall
[21:11:35] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:11:35] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall
[21:11:35] Camera(0) Gyro(0) SpO2(0) → Prediction: ✅ Non-Fall


c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\HP\Documents\DSGP\CM2603-Data_Science_Project-G20\fall_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
